# Bean Classification Project Proposal 

In [ ]:
#import libraries
install.packages("themis")
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 16)
library(readxl)

In [ ]:
library(themis)

# Introduction

This data set was extracted from the article written by Kolku and Ozkan, “Multiclass classification of dry beans using computer vision and machine learning techniques” (2020). Koluk and Ozkan (2020) explore the use of computer vision and machine learning techniques to classify dry beans into different classes. The goal of the study was to use machine learning algorithms to accurately classify the dry beans into their respective varieties based on the extracted features. This study is relevant for the food industry and agriculture, as it can potentially help in automating the classification process of dry beans, which is currently done manually and is time-consuming.

**Research Question**: Can we determine the bean type in an image based on its dimensional features?

This dataset is based on 13611 images of 7 types of individual dry beans with similar features. Each image was analyzed for 16 features of the bean (4 "shape factors", 12 structural/geometric features) (Kolku & Ozkan, 2020).

The most effective dimensional features in the dataset, are called: "Area", "Perimeter", "Major Axis Length", "Minor Axis Length", "Aspect Ratio", "Eccentricity", "Convex Area", "Equivalent Diameter", "Extent", "Solidity", "Roundness", "Compactness". We will be using the K-nearest neighbours classification to predict "Class" using the mentioned predictors. K-nearest neighbours find the K-closest data points to the input sample and assign the most common class label among them. The accuracy of the model can be improved by tuning the value of K.


Our variable of intetrest (Kolku & Ozkan, 2020).:
- **Area**:The area of a bean zone and the number of pixels within its boundaries.
- **Perimeter**: Bean circumference is defined as the length of its border.
- **Major Axis Length**: The distance of the longest line that can be drawn on the bean
- **Minor Axis Length**: The longest line that can be drawn on the bean while standing perpendicular to the axis
- **Aspect Ratio**: The relationship between the Major Axis Length and the Minor Axis Length
- **Eccentricity**: Eccentricity of the ellipse having the same moments as the region.
- **Convex Area**: Number of pixels in the smallest convex polygon that can contain the area of a bean seed.
- **Equivalent Diameter**: The diameter of a circle that has the same area as the bean area
- **Solidity**: The ratio of the pixels in the convex shell to those found in beans.
- **Extent**: The ratio of the pixels in the bounding box to the bean area.
- **Roundess**: Calculated with the following formula: (4piA)/(P^2). Where A is the area and P is the perimeter of the bean
- **Compactness**: Roundness of the bean in measurement

# Methods

**Load Data**: A zip file was loaded from the original web source (https). Then, tempfile () was used to ... 


In [ ]:
beanurl<-"https://archive.ics.uci.edu/ml/machine-learning-databases/00602/DryBeanDataset.zip"
temp <- tempfile()
download.file(beanurl,temp)
beanzip <- unzip(temp, "DryBeanDataset/Dry_Bean_Dataset.xlsx")
bean <- read_excel(beanzip)
bean

**Select variables & data wrangling & cleaning**: After exploring the data and doing research, we will select two variables for prediction: roundness and area. BECAUSE.... 
The "Class" column was also converted to a factor type for this classification model. 

In [ ]:
select_bean_var <- bean |>
                select("Area","roundness","Class")|>
                mutate(Class = as.factor(Class))
select_bean_var 

**Visualize plot**: all of the bean types, in relation to roundness and area, are plotted on the graph. 

In [ ]:
bean_plot <- select_bean_var |>
  ggplot(aes(x = Area, y = roundness, color = Class)) +
  geom_point(size = 0.2) +
  labs(x = "Area", 
       y = "Roundness",
       color = "Type") +
  theme(text = element_text(size = 12))
bean_plot

As shown on the graph, Barbunya has a lack of clustering that causes too many overlaps. This may result in an inaccurate model. Thus, we will remove the Class Barbunva. which will enable the KNN model to better distinguish between the remaining classes, ultimately improving the overall accuracy of the model.

In [ ]:
select_bean <- filter(select_bean_var, 
                      Class != "BARBUNYA")


In [ ]:
bean_plot2 <- select_bean |>
  ggplot(aes(x = Area, y = roundness, color = Class)) +
  geom_point(size = 0.2) +
  labs(x = "Area", 
       y = "Roundness",
       color = "Type") +
  theme(text = element_text(size = 12))
bean_plot2

We graph it again, and we can see that the clusters are more clean.

In [ ]:
##Creating training and testing dataset

set.seed(2022)
bean_split <- initial_split(select_bean, prop = 0.75, strata = Class)
bean_train <- training(bean_split)
bean_test <- testing(bean_split)

Graphing the training dataset in plot point

In [ ]:
#Graphing the training dataset

area_round_plot <- bean_train |>
  ggplot(aes(x = Area, y = roundness, color = Class)) +
  geom_point(size = 0.2) +
  labs(x = "Area", 
       y = "Roundness",
       color = "Type") +
  theme(text = element_text(size = 12))
area_round_plot

From the point plot of the area vs roundness observation above, we can see that there are big difference between area value and the roundness value, therefore, we will scale the data, which will be dealt with in `workflow()` 

In [ ]:
#Summarize training dataset
bean_class_dist <- bean_train |>
                group_by(Class)|>
                summarize(count = n()) |>
                mutate(percentage_dist = 100*count/nrow(bean_train))
bean_class_dist

The table, and bar chart show us the distribution of bean types in the training set

In [ ]:
bean_class_dist_plot <- bean_class_dist |>
                        ggplot(aes(x=Class, y = count))+
                        geom_bar(stat = "identity")+
                        labs(x= "Type of beans",
                             y = "Number of beans")+
                        ggtitle("Distribution of bean type in the training dataset")+
                        theme(text = element_text(size = 12))
bean_class_dist_plot

**Upsampling**: By looking at the table viewing the total number of datapoints for each type of beans, we can see that there are a big difference in number of datapoints between Dermason bean type with other type of beans. Therefore, we will upsample the training dataset so that each Class of bean has a voting power when it comes to the classfication of the testing dataset.

**Scaling:** all data is called to avoid features with large values dominating the decision process. Then the scaled data is plotted again.

In [ ]:
## Scalling all data:

bean_data_training_scaled_recipe <- recipe(Class ~., data = bean_train) |>
                        step_upsample(Class)|>
                        prep()

final_bean_data <- bake(bean_data_training_scaled_recipe, bean_train)

area_round_plot_scaled <- final_bean_data |>
  ggplot(aes(x = Area, y = roundness, color = Class)) +
  geom_point(size = 0.2) +
  labs(x = "Area (Standardized)", 
       y = "Roundness (Standardized)",
       color = "Type") +
  theme(text = element_text(size = 12))
area_round_plot_scaled

Summary of the data set: Next we will look at Statistical distribution of our chosen variables, in table and boxplot form. We calculated the basic statistics such as range and standard deviation for each feature, as well as determining the distribution. 


In [ ]:
#Statistical Distribution of predictor based on class
features_dist_by_class <- final_bean_data |>
                group_by(Class)|>
                summarize(max_area = max(Area, na.rm = TRUE),
                          min_area = min(Area, na.rm = TRUE),
                          std_dev_area = sd(Area, na.rm = TRUE),
                          max_roundness = max(roundness, na.rm = TRUE),
                          min_roundness = min(roundness, na.rm = TRUE),
                         std_dev_roundness = sd(roundness, na.rm = TRUE))
features_dist_by_class

In [ ]:
#Statistical Distribution in Features of varieties of dry bean
features_dist <- bean_train |>
            select(Area,roundness) |>
            pivot_longer(cols= Area:roundness,
                         names_to = "Features",
                         values_to = "values") |>
            group_by(Features) |>
            summarize(Mean = mean(values, na.rm = TRUE),
                      Min = min(values, na.rm = TRUE),
                      Max = max(values, na.rm = TRUE),
                     Std_Deviation = sd(values, na.rm = TRUE))
features_dist

In [ ]:
#Graphing area distribution for each type of bean
area_box_plot <- final_bean_data |>
                ggplot(aes(x = Class, y = Area))+
                geom_boxplot()+
                xlab("Type of beans")+
                ylab("Area")+
                ggtitle("Area Distribution for each type of bean")+
                coord_flip()
area_box_plot

In [ ]:
#Graphing box plot showing roundness distribution for each type of box plot
roundness_box_plot <- final_bean_data |>
                ggplot(aes(x = Class, y = roundness))+
                geom_boxplot()+
                xlab("Type of beans")+
                ylab("Roundess")+
                ggtitle("Roundness distribution for each type of bean")+
                coord_flip()
roundness_box_plot

# Building the Classification Model

In [ ]:
set.seed(2022) # DO NOT REMOVE

bean_report_recipe <- recipe(Class ~ ., data = final_bean_data) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())
bean_report_recipe

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

bean_vfold <- vfold_cv(final_bean_data, v = 10, strata = Class)
gridvals <- tibble(neighbors = seq(1,40,by=1))

knn_results <- workflow() |>
  add_recipe(bean_report_recipe) |>
  add_model(knn_spec) |>
  tune_grid(resamples = bean_vfold, grid = gridvals) |>
  collect_metrics() |>
  filter(.metric == "accuracy")

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 7)

# Plot k values against their respective accuracies and choose optimal k value
cross_val_plot <- knn_results |> 
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = "Neighbors", y = "Accuracy Estimate") +
    scale_x_continuous(breaks = seq(0,40, by = 1))
cross_val_plot

From the graph above we can see that with such a large dataset like the bean dataset, large number of neighbour is required to predict the data accurately. 

However, running cross validation for such a large dataset and large number of neighbors take a lot of time, and money, so we decided only estimate accuracies of different number of neighbors up until 45 neighbors. And it is clear that after 27 neighbours, the accuracies of different number of neighbors decreases, and the accuracies keeps decreasing until 40 neighbors, so we suspect that it will still decrease into 50 and 100 neighbors. 

From the graph we can see that the highest accuracies is around 25-27 neighbours. And we see that around 25 neighbors is good since the 26 neighbours, and 24 neighbours, the accuracies aren't that much different from the 25 neighbours. 

Visualizations of the analysis: a confusion matrix is shown below to help interpret the results and communicate the findings effectively.


In [ ]:
knn_best_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 25) |>
  set_engine("kknn") |>
  set_mode("classification")

bean_fit <- workflow() |>
  add_recipe(bean_report_recipe) |>
  add_model(knn_best_spec) |>
  fit(data = final_bean_data)
  
##(8)##
# Get the prediction column
bean_predictions <- predict(bean_fit, bean_test) |> 
    bind_cols(bean_test)

# Compare the accuracy of predictions to the true values in the test set
bean_acc <- bean_predictions |> 
    metrics(truth = Class, estimate = .pred_class) |> 
    select(.metric, .estimate) |> 
    head(1)

# Compare the predictions to the true values in a confusion matrix
bean_cm <- bean_predictions |> 
    conf_mat(truth = Class, estimate = .pred_class)
bean_acc
bean_cm

In [ ]:
#visual comparison of true class vs predicted class
library(ggplot2)
library(cowplot)

bean_predictions_plot <- bean_predictions |>
    ggplot(aes(x = Area, y = roundness, color = .pred_class)) +
    geom_point() +
    labs(x = "Area of Bean (Standardized)", y = "Roundness of Bean (standardized)", color = "Type of Bean") +
    ggtitle("Predictions of Bean Type")

bean_trueclass_plot <- bean_predictions |>
    ggplot(aes(x = Area, y = roundness, color = Class)) +
    geom_point() +
    labs(x = "Area of Bean (Standardized)", y = "Roundness of Bean (standardized)", color = "Type of Bean") +
    ggtitle("True Class of Beans")

plot_grid(bean_predictions_plot, bean_trueclass_plot, ncol = 2)

# Discussion

Summary of Results


What do we expect to find?
- With the help of a classification model, we expect to find a well-performing model that can predict the type of beans based on their roundness and area with high accuracy. On the visualization, we also expect to see a cluster for different types of bean in various colors.  

What impact could such findings have?
- This classification model can impact the food and agriculture industry. An automated system can be created to classifiy the type of bean based on their physical appearance. This is espcially useful when food and agriculture companies collect different types of beans together, dry them, and want to package them based on different types. The model we built can help to streamline the service, gain quality control and reduce the time for people to manually separate the beans. 

What future questions could this lead to?
- Can other variables such as major axis length or the perimeter be good predictors of the type? And are they better predictors than roundness and area?
- Can the classification model be used for different types of crops such as rice? 

###### References:
- Krzywinski, Martin, and Naomi Altman. "Visualizing samples with box plots: use box plots to illustrate the spread and differences of samples." Nature Methods, vol. 11, no. 2, Feb. 2014, pp. 119+. Gale OneFile: Health and Medicine, link.gale.com/apps/doc/A361242515/HRCA?u=ubcolumbia&sid=bookmark-HRCA&xid=0db0fe06. Accessed 11 Mar. 2023.

- Koklu, M., & Ozkan, I. A. (2020). Multiclass classification of dry beans using computer vision and machine learning techniques. Computers and Electronics in Agriculture, 174, 105507. https://doi.org/10.1016/j.compag.2020.105507